<a href="https://colab.research.google.com/github/lblum95/AML/blob/master/task1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AML first project

## Data ingestion

### Connect to drive

In [10]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/My Drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Import all data libraries

In [11]:
import pandas as pd
from pandas_profiling import ProfileReport
import numpy as np

### Import data

In [12]:
x_train = pd.read_csv("data/X_train.csv",index_col=0,header = 0)
y_train = pd.read_csv("data/y_train.csv",index_col=0,header = 0)
x_test = pd.read_csv("data/X_test.csv",index_col=0,header = 0)

### Data cleaning


Outlier clipping

quantile ignores nans :)

In [13]:
percentile=0.05
#quantile ignores nans :)
print(x_train.shape)
lower=x_train.quantile(percentile, axis=0, interpolation='lower')
higher=x_train.quantile(1-percentile, axis=0, interpolation='higher')
print(lower.shape)
print(higher.shape)
train=x_train.clip(lower,higher,axis=1)
print(train.shape)

(1212, 832)
(832,)
(832,)
(1212, 832)




Fill mean in NaNs

In [14]:
#fill NaNs with mean
train=train.apply(lambda x: x.fillna(x.mean()),axis=0)

Normalize data

In [15]:
from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(train)
test=scaler.transform(train)
test

array([[ 1.03395957e+00, -1.72388823e+00,  6.95963777e-01, ...,
        -5.24358916e-01, -1.71844251e+00, -7.03238121e-01],
       [ 1.77519194e+00,  1.45780409e+00,  3.96418573e-10, ...,
        -1.62112772e+00, -4.88124515e-01,  9.97882048e-02],
       [ 1.22526942e+00,  1.39000401e+00, -4.38707639e-01, ...,
        -1.13899489e+00, -1.83947835e+00, -2.81512241e-01],
       ...,
       [-9.04759519e-01, -1.63401639e+00,  3.23005914e-02, ...,
        -1.59444405e+00,  1.81870577e+00,  1.30501687e-01],
       [-1.05993919e-14, -1.22064009e+00,  1.42144559e+00, ...,
        -8.71659095e-02,  5.86512874e-01, -1.68631832e+00],
       [-1.13927439e+00, -6.58484639e-01,  2.53751452e-01, ...,
        -7.64400524e-01, -1.72872580e-14, -1.76454689e-01]])

## Feature selection

### import all libraries

In [16]:
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.feature_selection import RFECV
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
import numpy as np

### Fiddle down to 200 features

In [17]:
kbest=SelectKBest(f_regression, k=200)
train = kbest.fit_transform(test, np.ravel(y_train))
print(train.shape)

(1212, 200)


/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:299: RuntimeWarning: invalid value encountered in true_divide
  corr /= X_norms
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:1912: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


### RFECV with linear SVR 

In [25]:
linear = SVR(kernel="linear")
rfe_linear = RFECV(linear, step=5, cv=5)
train_linear = rfe_linear.fit_transform(train, np.ravel(y_train))
print(train_linear.shape)

(1212, 40)


Test SVR

In [26]:
train_x, val_x, train_y, val_y = train_test_split(train_linear, y_train, test_size=0.2, random_state=36)
linear = SVR(kernel="linear")
clf = linear.fit(train_x, np.ravel(train_y))
print(clf.score(val_x, np.ravel(val_y))) #already correct r2 metric :)

0.5074965285156073


In [27]:
tree = DecisionTreeRegressor(random_state=0)
rfe_tree = RFECV(tree, step=5, cv=5)
train_tree = rfe_tree.fit_transform(train, np.ravel(y_train))
print(train_tree.shape)

(1212, 25)


In [29]:
train_x, val_x, train_y, val_y = train_test_split(train_tree, y_train, test_size=0.2, random_state=36)
tree = DecisionTreeRegressor(random_state=0)
clf = tree.fit(train_x, np.ravel(train_y))
print(clf.score(val_x, np.ravel(val_y))) #already correct r2 metric :)

0.18383500900706728


In [22]:
from IPython.display import display, Markdown